
## Upload YouTube Videos to the Reading Material Folder Manually


In this notebook, we will take URLs of videos, convert them to mp3s, upload them to Google Drive, trash them, and delete them.


We will start by ensuring that all the necessary python libraries are installed.

In [ ]:

import sys

print('conda update --all --yes --prefix "{}"'.format(sys.prefix))

In [ ]:

!{sys.executable} -m pip install --upgrade youtube_dl

In [ ]:

import os

# https://github.com/ytdl-org/youtube-dl/issues/20758
# Just replaced 'token' to 'account_playback_token' in line 1674 of extractor/youtube.py
# All seems to work.
for sub_dir, dirs_list, files_list in os.walk(r'C:\Users\dev\Anaconda3\Lib\site-packages\youtube_dl'):
    for file_name in files_list:
        if 'youtube.py' == file_name:
            print(os.path.join(sub_dir, file_name))

In [ ]:

!youtube-dl --version

In [ ]:

!{sys.executable} -m pip install --upgrade PyDrive


Next, we will prove to ourselves that we have ffmpeg correctly installed.

In [ ]:

# https://github.com/adaptlearning/adapt_authoring/wiki/Installing-FFmpeg
!ffmpeg -version


Performing authentication with Google Drive requires we keep our client id and consumer secret in a json file. Keep your secrets off the internet!

In [ ]:

!start %windir%\explorer.exe "C:\Users\dev\Documents\repositories\notebooks\Miscellaneous\json"


We also make sure line 286 of pydrive's files.py file reads <code>self.content.close()</code> before we run the trash cell.

In [ ]:

file_path = 'C:\Users\dev\Anaconda3\Lib\site-packages\pydrive\files.py'
!"C:\Program Files\Notepad++\notepad++.exe" "{file_path}"


Before we start the download process, let's go over setting a few variables:

* **The downloads folder:** This is where all the messy work will be done. All mp3 files and any intermediate files will be deleted out of this folder by the end.
* **The URL list:** This is where we place our YouTube video URLs. It doesn't matter if you remove the <code>time_continue=</code> parameter in the URL, the mp3 will just start at zero seconds anyway.
* **The target folder ID:** This is the folder in Google Drive where our uploads will end up.

In [19]:

import youtube_dl

downloads_folder = r'C:\Users\577342\Downloads\\'
youtube_url_list = ['https://www.youtube.com/watch?v=oCK7td5uWsU',
                    'https://www.youtube.com/watch?v=5zXLuXOV7Po&feature=share']

Now we are ready to start the download process by running <code>youtube_dl</code> on our <code>youtube_url_list</code>.

In [20]:
ydl_opts = {
    'format': 'bestaudio/best',
    'nocheckcertificate': False,
    'outtmpl': downloads_folder+youtube_dl.DEFAULT_OUTTMPL,
    'postprocessors': [{'key': 'FFmpegExtractAudio',
                       'preferredcodec': 'mp3',
                       'preferredquality': '192'}],
    'verbose': True,
    }
with youtube_dl.YoutubeDL(ydl_opts) as ydl:
    try:
        result = ydl.download(youtube_url_list)
    except Exception as e:
        print(e)
print('Conversion completed.')

[debug] Encodings: locale cp1252, fs utf-8, out UTF-8, pref cp1252
[debug] youtube-dl version 2019.04.17
[debug] Python version 3.6.8 (CPython) - Windows-10-10.0.17763-SP0
[debug] exe versions: ffmpeg N-90173-gfa0c9d69d3, ffprobe N-90173-gfa0c9d69d3, phantomjs 2.1.1
[debug] Proxy map: {}


[youtube] oCK7td5uWsU: Downloading webpage
[youtube] oCK7td5uWsU: Downloading video info webpage
[debug] Invoking downloader on 'https://r1---sn-p5qlsnd6.googlevideo.com/videoplayback?id=o-AF0ThHXyzEnumU3kDaQRyTLNEmTVUmXU_BScGhoAKsbI&itag=140&source=youtube&requiressl=yes&mm=31%2C26&mn=sn-p5qlsnd6%2Csn-vgqsknel&ms=au%2Conr&mv=m&pl=32&ei=lsPKXN7pJILzwQG9lYSgCA&initcwndbps=1942500&mime=audio%2Fmp4&gir=yes&clen=19044142&dur=1199.031&lmt=1518995002732964&mt=1556792050&fvip=1&keepalive=yes&c=WEB&ip=2600%3A6c64%3A657f%3Af499%3A0%3Aab34%3A2e17%3A2a78&ipbits=0&expire=1556813814&sparams=ip%2Cipbits%2Cexpire%2Cid%2Citag%2Csource%2Crequiressl%2Cmm%2Cmn%2Cms%2Cmv%2Cpl%2Cei%2Cinitcwndbps%2Cmime%2Cgir%2Cclen%2Cdur%2Clmt&signature=EA1083CDB8B4541BAC054181B9107AB235AD1EC4.91FB6244C8A7249F48A520C31571B24D5C60B026&key=yt8&ratebypass=yes'
[download] Destination: C:\Users\577342\Downloads\The Tainter Plus Model _ Deep Code Experiment - Episode 5-oCK7td5uWsU.m4a
[download] 100% of 18.16MiB in 00:03        


Next, we authenticate with Google by running the builtin web server conventiently provided.

In [21]:

from pydrive.auth import GoogleAuth

# The URL that gets displayed here is not suitable for public consumption
GoogleAuth.DEFAULT_SETTINGS['client_config_file'] = r'../json/client_secrets.json'
gauth = GoogleAuth()
gauth.LocalWebserverAuth()

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=756414784841-0nkbgkrice8gae3nc4ojafoukr8kb38e.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.



Before you run the next cell, in the online web page of Google Drive, bring up the share dialog box of the folder you are uploading to and copy its folder id into the <code>tgt_folder_id</code> string. If you right-click on the folder and select **Share...** and then click **Get shareable link**, you will see something like:

https://drive.google.com/drive/folders/1syfUx6jukbW1CWIEy8xoM9veGGr5MBUx?usp=sharing

It's that **1syfUx6jukbW1CWIEy8xoM9veGGr5MBUx** that you need to copy.

In [22]:

from pydrive.drive import GoogleDrive

# Create GoogleDrive instance with authenticated GoogleAuth instance
drive = GoogleDrive(gauth)

# ID of the "Reading Material" folder
tgt_folder_id = '1syfUx6jukbW1CWIEy8xoM9veGGr5MBUh'


Next, we will upload all the mp3s in our download folder.

In [23]:

import os

# Upload all mp3s from the Downloads folder to Google Drive's "Reading Material" folder
gfile_dict = {}
for subdir, dirs, files in os.walk(downloads_folder):
    for src_file in files:
        if src_file.endswith('.mp3'):
            src_path = os.path.join(subdir, src_file)
            gfile_dict[src_file] = drive.CreateFile({'title':src_file, 'mimeType':'audio/mp3',
                                                     'parents': [{'kind': 'drive#fileLink',
                                                                  'id': tgt_folder_id}]})

            # Read mp3 file and set it as a content of this instance
            gfile_dict[src_file].SetContentFile(src_path)
            
            # Upload the file
            try:
                gfile_dict[src_file].Upload()
                print('Uploaded %s (%s)' % (gfile_dict[src_file]['title'],
                                            gfile_dict[src_file]['mimeType']))
            except Exception as e:
                print('Upload failed for %s (%s): %s' % (gfile_dict[src_file]['title'],
                                                         gfile_dict[src_file]['mimeType'], e))
print('Upload completed.')

Uploaded The Tainter Plus Model _ Deep Code Experiment - Episode 5-oCK7td5uWsU.mp3 (audio/mp3)
Uploaded The War Against Video Games-5zXLuXOV7Po.mp3 (audio/mp3)
Upload completed.



Download the mp3s to <a href='http://www.voicedream.com/support/user-manual/#file'>Voice Dream</a> before you run the trash cell (below).

In [24]:

import os

# Trash all mp3s from Google Drive's "Reading Material" folder
for src_file in gfile_dict.keys():

    # Trash mp3 file
    try:
        gfile_dict[src_file].Trash()
        print('Trashed %s (%s)' % (gfile_dict[src_file]['title'],
                                   gfile_dict[src_file]['mimeType']))
    except Exception as e:
        print('Trash failed for %s (%s): %s' % (gfile_dict[src_file]['title'],
                                                gfile_dict[src_file]['mimeType'], e))
    
print('Trashing completed.')

Trashed The Tainter Plus Model _ Deep Code Experiment - Episode 5-oCK7td5uWsU.mp3 (audio/mp3)
Trashed The War Against Video Games-5zXLuXOV7Po.mp3 (audio/mp3)
Trashing completed.



Lastly, we run the code that will delete all the mp3s out of our downloads folder.

In [25]:

import os

# Delete all mp3s in the Downloads folder
for src_file in gfile_dict.keys():
    src_path = os.path.join(downloads_folder, src_file)

    # Delete the file
    try:
        os.remove(src_path)
        print('Deleted %s (%s)' % (src_file,
                                   gfile_dict[src_file]['mimeType']))
    except Exception as e:
        print('Failed to delete %s (%s): %s' % (src_file,
                                                gfile_dict[src_file]['mimeType'], e))
print('Deleting completed.')

Deleted The Tainter Plus Model _ Deep Code Experiment - Episode 5-oCK7td5uWsU.mp3 (audio/mp3)
Deleted The War Against Video Games-5zXLuXOV7Po.mp3 (audio/mp3)
Deleting completed.


In [26]:

from IPython.display import HTML

rm_url = 'https://drive.google.com/drive/u/0/folders/{}'.format(tgt_folder_id)
message_str = 'open up your Google Drive target folder'
rm_link = '<a href="{}" target="_blank">{}</a>'.format(rm_url, message_str)
message_str = 'If you want to, you can {} and'.format(rm_link)
message_str += ' check if everything got deleted.'
HTML(message_str)

In [27]:

!"C:\Program Files (x86)\Google\Chrome\Application\chrome.exe" {rm_url}